In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test = pd.read_csv('test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


数值型和类别型分开处理

In [4]:
test.dtypes['Id']

dtype('int64')

In [5]:
num_columns = []
cate_columns = []
for column in test.columns:
    if test.dtypes[column] != np.dtype('object'):
        num_columns.append(column)
    else:
        cate_columns.append(column)
print(num_columns)
print(cate_columns)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'Gar

In [6]:
label = train.pop('SalePrice')

填充缺失

In [7]:
# 数值型用中值填充
for column in num_columns:
    train[column] = train[column].fillna(train[column].median())
    test[column] = test[column].fillna(test[column].median())

# # 类别型用最多的填充 # 线上0.13488
# for column in cate_columns:
#     train[column] = train[column].fillna(train[column].mode())
#     test[column] = test[column].fillna(test[column].mode())
    
# 类别型填充'NaN' # 线上0.13436
for column in cate_columns:
    train[column] = train[column].fillna('NaN')
    test[column] = test[column].fillna('NaN')

类别型哑变量处理

In [8]:
data = pd.concat([train,test],axis=0) #训练集要和测试集放一起
for column in cate_columns:
    
    t = pd.get_dummies(data[column],prefix=column)
    train = pd.concat([train,t[:len(train)]],axis=1)
    train.drop(column,axis=1,inplace=True)
    
    test = pd.concat([test,t[len(train):]],axis=1)
    test.drop(column,axis=1,inplace=True)

回归走起

In [9]:
from sklearn.linear_model import Lasso,LinearRegression,Ridge,ElasticNet,TheilSenRegressor,HuberRegressor,RANSACRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools

C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.33, random_state=42)

In [11]:
regs = [
    ['Lasso',Lasso()],
    ['LinearRegression',LinearRegression()],
    ['Ridge',Ridge()],
    ['ElasticNet',ElasticNet()],
    ['TheilSenRegressor',TheilSenRegressor()],
    ['RANSACRegressor',RANSACRegressor()],
    ['HuberRegressor',HuberRegressor()],
    ['SVR',SVR(kernel='linear')],
    ['DecisionTreeRegressor',DecisionTreeRegressor()],
    ['ExtraTreeRegressor',ExtraTreeRegressor()],
    ['AdaBoostRegressor',AdaBoostRegressor(n_estimators=150)],
    ['ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=150)],
    ['GradientBoostingRegressor',GradientBoostingRegressor(n_estimators=150)],
    ['RandomForestRegressor',RandomForestRegressor(n_estimators=150)],
    ['XGBRegressor',XGBRegressor(n_estimators=150)],
]

In [ ]:
preds = []
for reg_name,reg in regs:
    print(reg_name)
    reg.fit(X_train,y_train)
    y_pred = reg.predict(X_test)
    if np.sum(y_pred<0) > 0:
        print('y_pred have',np.sum(y_pred<0),'negative values, we fill it with np.median(y_pred)')
        y_pred[y_pred<0] = np.median(y_pred)
    score = np.sqrt(mean_squared_error(np.log(y_test),np.log(y_pred)))
    preds.append([reg_name,y_pred])

Lasso


C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LinearRegression
Ridge
ElasticNet
TheilSenRegressor


In [ ]:
final_results = []
for comb_length in range(1,len(regs)+1):
    print('Model Amount :',comb_length)
    results = []
    for comb in itertools.combinations(preds,comb_length):
        pred_sum = 0
        model_name = []
        for reg_name,pred in comb:
            pred_sum += pred
            model_name.append(reg_name)
        pred_sum /= comb_length
        model_name = '+'.join(model_name)
        score = np.sqrt(mean_squared_error(np.log(y_test),np.log(pred_sum)))
        results.append([model_name,score])
    results = sorted(results,key=lambda x:x[1])
    for model_name,score in results:
        print(model_name,score)
    print()
    final_results.append(results[0])

In [ ]:
final_results = sorted(final_results,key=lambda x:x[1])
for model_name,score in final_results:
    print(model_name,score)

最终输出

In [ ]:
[b for b in zip(itertools.count(),[a[0] for a in regs])]

In [ ]:
pred = np.mean(list(map(lambda x:regs[x][1].predict(test),[0,12,14])),axis=0)

In [ ]:
sub = pd.DataFrame({'Id':test['Id'],'SalePrice':pred})
sub.to_csv('submission_Universe_fillNaN.csv',index=None)